In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


/Users/aayeshaaayesha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# buyer-side sentiments
import pandas as pd
df = pd.read_excel('buyer_sentiments.xlsx')
df = df.drop(index=0)
df

,UID,Name,interface1b,interface2b,interface3b,function1b,function2b,accomp1b,accomp2b
1,82,Alexander,Yes,Yes,Yes,Yes,Yes,Yes,Yes
2,85,Emirhan,It is really useful to send people for voting ...,It was easy but there might be some informatio...,it was easy,it just took a few click to forward.,"I is useful because, I value the other people'...","I really find it useful, before i did not give...","In my opinion, there should be more photo for ..."
3,83,Mustafa,It was okay,Was easy,Easy to understand,Yeah,It need some enhancements,No that much -o,No that much
4,84,Enxhi,The wishlist needs to have a button where I ca...,"It was easy to find, but there is no reason fo...",The list of items were easy to understand.,The only problem with sharing is that I had to...,The intent information was not really useful a...,"Yes, this is a very helpful feature. I would u...",It is useful for the person getting the feedba...
5,123,Sarah,"Yes, I like the overview. The pictures when gi...","yes, although it was not 100% clear that you a...","easy to understand, clear communication","yes, but not via Telegram/Facebook Messenger a...",Yes,"yes, definitely",yes
...,...,...,...,...,...,...,...,...,...
70,131,Florian Haubenberger,yes the wishlist view was a good feature but i...,yes the link sharing was easy but it got quite...,it was very easy to understand the ranking but...,"yes it was very smooth, just forward the link ...","yes definitely, it is important to know the oc...",yeah i think its a great feature since you can...,"not in the telegram bot, the intent was not di..."
71,135,Yannik,There should be a feature to empty it. Except ...,Yes,"Yes, just a simple order. The design that any ...",Yes,"Yes, because the occasion influences appropria...","Yes, it is a nice feature. I would probably us...","Yes, because the occasion influences appropria..."
72,137,Roman,"yes, but it could be better if you don't have ...","relatively, yes","yes, seems so","yes, it was shared without any problems","yes, though it's not the only criteria, which ...","relatively useful, though the choice of items ...",yes
73,138,Kristian,Yes,Yes,Easy in terms of the results. Difficult to und...,Yes,Yes,Yes,"Better quality, and some angles would be bette..."


In [4]:
# buyer-side sentiments
from statistics import mean
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))

[0.42025307, 0.19173831, 0.38800865]
[0.4438927, 0.16371614, 0.39239118]
[0.36161235, 0.1916105, 0.44677716]
[0.41644406, 0.11793827, 0.46561766]
[0.363592, 0.13089716, 0.50551087]
[0.45214352, 0.12129267, 0.42656383]
[0.37715015, 0.1385469, 0.48430294]


In [5]:
# buyer-side sentiments
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.4085860252380371
 interface negative sentiment is 0.18235498666763306
 interface neutral sentiment is 0.40905898809432983
 function positive sentiment is 0.3900180459022522
 function negative sentiment is 0.12441772222518921
 function neutral sentiment is 0.485564261674881
 accomplish positive sentiment is 0.41464683413505554
 accomplish negative sentiment is 0.12991978228092194
 accomplish neutral sentiment is 0.45543336868286133


In [24]:
# friend-side sentiments
df = pd.read_excel('friend_sentiment.xlsx')
df = df.drop(index=0)
df

,Unnamed: 0,Unnamed: 1,interface1f,interface2f,function1f,function2f,accomp1f
1,83,Mustafa,Maybe consider adding more than one picture fo...,I think the point of all friends can participa...,No comment,They were useful but are not showing according...,Yes.
2,82,Alexander,It was ok,Drag and drop was handy,Why some of them where optional,To know what the items will be used for,I think this was a good solution
3,85,Emirhan,There might be more detailed photo like qualit...,We have to slide items instead of clicking.,"When, a person send more than one message to v...",I did not find useful intent information excep...,I am satisfied with the way of voting
4,83,Mustafa,I think adding more pics for each item will be...,I don't know.,I don't know,Good enough,It was good
5,152,Zuzanna,"The view wasn't sufficient, I would add more p...",The heart for the best item is cool,why is there so much room between the name and...,"There is not enough info on the items, the clo...","Instead of dragging the items, I would rather ..."
...,...,...,...,...,...,...,...
70,114,Anastasia,"I want an improved view, where i can see how t...",Ranking. I think this kind of ranking was a go...,Everything was clear to me.,"yes, it was pretty useful, therefore i could b...","Yes, I am satisfied"
71,131,Florian Haubenberger,"the view was bad tbh, the pictures were croppe...",the drag and drop worked fine,"why the css was so bad, there are many ways to...",the intent was written in the headline so i kn...,i think it would be better to get e.g. 1 point...
72,135,Yannik,Was sufficient,Drag and drop was nice.,How many items should be ranked. But it was cl...,It was useful because the occasion influences ...,The pictures could have been bigger. Always ch...
73,138,Kristian,More angles would be great.,"I did not have to vote all of them, which is g...",The interface was okay. Just missing some info...,It was useful,I am quite happy


In [25]:
# friend-side sentiments
from statistics import mean
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))

[0.4365694, 0.13181451, 0.4316161]
[0.47487113, 0.15746436, 0.36766455]
[0.15504773, 0.27404496, 0.5709073]
[0.4029654, 0.16753386, 0.42950073]
[0.396971, 0.19006063, 0.41296837]


In [26]:
# buyer-side sentiments
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 1:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 2 or j == 3:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 3:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.4557202458381653
 interface negative sentiment is 0.14463943243026733
 interface neutral sentiment is 0.3996403217315674
 function positive sentiment is 0.27900657057762146
 function negative sentiment is 0.22078940272331238
 function neutral sentiment is 0.5002040266990662
 accomplish positive sentiment is 0.3969709873199463
 accomplish negative sentiment is 0.19006063044071198
 accomplish neutral sentiment is 0.41296836733818054


In [27]:
# personalised sentiments
df = pd.read_excel('person_sentiment.xlsx')
df = df.drop(index=0)
df

,Unnamed: 0,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
1,119,Fung Yee,the ranking is easy to apply,the bot has pictures of the clothes you could ...,"you need to press ""start"" button and share to ...",low efficiency,no,actually not. i found out the recommendations ...,no. the recommendations from their feedback is...,the ranking result of mine and theirs is not c...
2,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
3,144,Antal,It was pretty hard to normally use the bot due...,The interface was not intuitive and therefore ...,"Chat system in telegram, sending vote which pr...",Using the bot provided actually even more work...,"The filtering was bad, reload of the page duri...",There was some help for aggregating the votes ...,To some extent however I still highly used my ...,Sometimes it was correct but for the most part...
4,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
5,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
...,...,...,...,...,...,...,...,...,...,...
69,135,Yannik,Instructions were clear.\n,Overview of shopping cart.,Process of giving recommendation had to be don...,Okay.,"Yes, the telegram interface makes it a little ...",Reasonable amount. The reasons behind the rank...,A little. But without reasoning not really.,Mostly accurate.
70,137,Roman,"It was clear, what to expect from each step",Generated pictures are easy to comprehend also...,"It was a bit unclear, whether feedback is prov...","It was quite efficient, no technical issues",It was fine,Suggestions aligned with something I would lik...,Not entirely as I would say I have other facto...,It was fine
71,138,Kristian,.,.,.,.,.,.,.,.
72,139,Rebeka,It is easy to ask friends for recommendations ...,The way items can be sorted when giving recomm...,It's hard to keep track of my wishlists and al...,It could be more efficient with a better navig...,"Yes, the navigation is not smooth. I click on ...","I got all the help I could possibly get, as al...",I wouldn't say the bot helped me much. My frie...,"Out of 5, only one recommendation was complete..."


In [28]:
# personalised experience
from statistics import mean
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))

[0.38826114, 0.113087356, 0.49865147]
[0.384724, 0.13972117, 0.47555482]
[0.071984135, 0.39157933, 0.53643656]
[0.44411975, 0.2065861, 0.3492942]
[0.24322976, 0.317986, 0.43878424]
[0.4611125, 0.14579989, 0.3930876]
[0.33064064, 0.17976606, 0.4895933]
[0.34315363, 0.15782577, 0.4990206]


In [29]:
# personalised experience
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.281656414270401
 interface negative sentiment is 0.21479594707489014
 interface neutral sentiment is 0.5035476088523865
 function positive sentiment is 0.34367474913597107
 function negative sentiment is 0.26228606700897217
 function neutral sentiment is 0.39403921365737915
 accomplish positive sentiment is 0.3783022463321686
 accomplish negative sentiment is 0.16113057732582092
 accomplish neutral sentiment is 0.4605671763420105


In [30]:
# social circle-level sentiments
df = pd.read_excel('social_sentiment.xlsx')
df = df.drop(index=0)
df

,Unnamed: 0,Unnamed: 1,interface1s,interface2s,interface3s,function1s,function2s,accomp1s,accomp2s,accomp3s
1,140,Christoph,None because the interface was terrible so the...,None the interface was not really helpful. As ...,Like the chat system in telegram and all the h...,There was no efficiency just more work then se...,The page reloaded nearly every interaction and...,Dont know because í only got the results from ...,Not really,Didn't fit well
2,83,Mustafa,The selection of different categories or the s...,The wishlist.,I don't know,"Was good, just it take time and refresh the pa...",Yeah as it mentioned above and also when I wan...,There were helpful. I do reconsider some of or...,Not that much honestly,Almost the same
3,100,Kasra,give feedback button,screenshot of the clothes,the sharing button,it was not very efficient. too many steps invo...,it was smooth,medium levels of help,yes it helps,it was very similar
4,126,Mirjana,The availability of the function to share con...,Receiving diverse recommendations and insights...,The intent modification feature was not very c...,In regards to the efficiency of the Bot during...,In regards to the smoothness of clicking diffe...,In regards to how helpful friends' feedback wa...,In regards to whether the Bot recommendations ...,In regards to how we perceived the Bot recomme...
5,102,Theresa,easy to arrange the feedback,easy to share the wishlist with your friends a...,"unsubscribe button, no explicit info on how ma...",low - ok,we had to click on every picture separately an...,"depends on the person, some had 2nd thoughts a...",can be a confirmation or dissuasion to buy spe...,only in the two extreme cases: everyone encour...
6,86,Maximilian,"The ""start"" and ""show results"" buttons.",The result view.,"The results were unclear, because some ranking...","Sometimes slow, but ok.","Yes, sometimes it takes time to show the results.",Barely any help.,No it wasn't able.,"Often the group had a favorite item, which all..."
7,153,Nils,"Telegramm API, Add to wishlist button, Ranking","nice colors, good button form",Click for feedback -> how to get the feedback ...,pretty efficient for what it was supposed to do,The loading times could be faster. And the wor...,Yes and No 50/50 -> because some of us only as...,No 2/2 Yes,For some it was close for some it was far appa...
8,149,Wei,Share button is easy to use. Download version ...,Switching between web and bot is attractive. R...,Hard to distinguish between each others recomm...,"It is good, but the process is not good enough","No, it is working well\n","It is helping a lot, ranking orders same as ot...",It combines others recommendation and make a h...,Quit same as friends' recommendations
9,109,Florian,Adding to wishlist was easy,Four squares that displayed clothing,The way it works on two telegram chats,quite confusing but fast,no it was fast,"Not much, I already made up my mind before",No I made up my mind before,Roughly similar
10,113,Maximilian Höller,"very nice was that, we did not have to make a ...",we think that it was very attractive to use th...,sometimes the rating was unclear (it seemed li...,we did not have any performance issues,not really,"not so much, because the taste of the differen...","in some cases yes, in some cases no","once again, we think that in some cases it was..."


In [31]:
from statistics import mean
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))



[0.50634676, 0.07869195, 0.41496128]
[0.44228107, 0.08211879, 0.4756001]
[0.11166915, 0.42447916, 0.4638517]
[0.2550883, 0.34558165, 0.39933002]
[0.17286554, 0.29832616, 0.5288083]
[0.34464476, 0.18477724, 0.47057799]
[0.27148956, 0.17371897, 0.55479145]
[0.22381432, 0.18146476, 0.5947209]


In [32]:
# social-circle experience
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.3534323275089264
 interface negative sentiment is 0.19509662687778473
 interface neutral sentiment is 0.4514710307121277
 function positive sentiment is 0.2139769196510315
 function negative sentiment is 0.3219538927078247
 function neutral sentiment is 0.4640691578388214
 accomplish positive sentiment is 0.27998286485671997
 accomplish negative sentiment is 0.17998699843883514
 accomplish neutral sentiment is 0.5400301218032837
